# Caracteristica MOD09A1.006 RS google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"},name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Coleccion MODIS09

In [6]:
# Coleccion de MODIS SR
MODIS_RS = ee.ImageCollection("MODIS/006/MOD09A1")\
              .filterDate('2019-01-01','2019-12-31')\
              .filterBounds(zona_ee)

In [7]:
# Conteo de imagenes
count = MODIS_RS.size()
print("Cantidad de imagenes MODIS_RS:", count.getInfo())

Cantidad de imagenes MODIS_RS: 46


In [10]:
# Imprimir la lista de ID Imagenes
ID_MODIS_RS = MODIS_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_MODIS_RS)

['2019_01_01', '2019_01_09', '2019_01_17', '2019_01_25', '2019_02_02', '2019_02_10', '2019_02_18', '2019_02_26', '2019_03_06', '2019_03_14', '2019_03_22', '2019_03_30', '2019_04_07', '2019_04_15', '2019_04_23', '2019_05_01', '2019_05_09', '2019_05_17', '2019_05_25', '2019_06_02', '2019_06_10', '2019_06_18', '2019_06_26', '2019_07_04', '2019_07_12', '2019_07_20', '2019_07_28', '2019_08_05', '2019_08_13', '2019_08_21', '2019_08_29', '2019_09_06', '2019_09_14', '2019_09_22', '2019_09_30', '2019_10_08', '2019_10_16', '2019_10_24', '2019_11_01', '2019_11_09', '2019_11_17', '2019_11_25', '2019_12_03', '2019_12_11', '2019_12_19', '2019_12_27']


In [12]:
ID_MODIS_RS[26]

'2019_07_28'

## Seleccion de una imagen

In [14]:
# Importar imagen MODIS 09
MODIS_img = ee.Image("MODIS/006/MOD09A1" + "/" + ID_MODIS_RS[26])

In [15]:
# Nombre de las bandas
print(MODIS_img.bandNames().getInfo())

['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'QA', 'SolarZenith', 'ViewZenith', 'RelativeAzimuth', 'StateQA', 'DayOfYear']


In [16]:
# Renombrar los nombres bandas
bandas = ["sur_refl_b01","sur_refl_b02","sur_refl_b03","sur_refl_b04", "sur_refl_b05", "sur_refl_b06","sur_refl_b07"]
newbandas = ["B1","B2","B3","B4","B5","B6","B7"]
MODIS_img = MODIS_img.select(bandas, newbandas)
MODIS_img.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

## Reproyectar

In [17]:
# Reproyectar segun zona de estudio
MODIS_img_py = MODIS_img.reproject(crs = "EPSG:4326", scale = 500)

In [18]:
# Consultar Proyeccion
MODIS_img_py.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:4326',
 'transform': [0.004491576420597608, 0, 0, 0, -0.004491576420597608, 0]}

## Factor de escala

In [19]:
# Multiplicar factor de escala
MODIS_img_py1 = MODIS_img_py.multiply(0.0001)

In [20]:
# Simbologia de MODIS
viz = {
    'bands': ['B7','B2','B1'],
    'min': 0,
    'max': 0.3,
    'gamma': 1.5
}

In [21]:
# Visualizar 
Map.addLayer(MODIS_img_py1,viz,name = "MODIS RS")
Map

Map(bottom=281405.0, center=[-12.908867400877059, -69.99458312988283], controls=(WidgetControl(options=['posit…

## Recortar segun zona estudio

In [22]:
# Recortar zona estudio
MODIS_img_py1_clip = MODIS_img_py1.clip(zona_ee)

In [24]:
# Visualizar 
Map.addLayer(MODIS_img_py1_clip, viz,name = "MODIS RS Clip")
Map

Map(bottom=2444.0, center=[-8.407168163601076, -58.62304687500001], controls=(WidgetControl(options=['position…

## Cambiar proyeccion UTM

In [25]:
# Cambiar la proyeccion
MODIS_img_py1_clip_utm = MODIS_img_py1_clip.reproject(crs = "EPSG:32719", scale = 500)

## Exportar Imagen calibrado

In [26]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [27]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [28]:
# Exportar imagen directorio
geemap.ee_export_image(MODIS_img_py1_clip_utm, filename="MODIS_img_py1_clip_utm.tif", scale=500, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\MODIS_img_py1_clip_utm.tif


In [29]:
# Exportar en google drive
geemap.ee_export_image_to_drive(MODIS_img_py1_clip_utm, description='MODIS_img_py1_clip_utm', folder='GEE_Python', region=geometria, scale=500)

Exporting MODIS_img_py1_clip_utm ...
